In [ ]:
!pip install rank-bm25

In [ ]:
import re
import math
from rank_bm25 import BM25Okapi
import pickle
from tqdm.notebook import tqdm
from FibCoder import FibCoder
import os
import codecs
from multiprocessing.dummy import Pool as ThreadPool 
from multiprocessing.dummy import Lock as ThreadLock 
from collections import defaultdict
from multiprocessing.dummy import Value as ThreadValue
import functools

In [ ]:
def illustration(func):
    """
    Распаралеливание выкачки страниц.
    """
    mutex = ThreadLock()
    n_thread = ThreadValue('i',0)
    @functools.wraps(func)
    def wrapper(*args, **argv):
        result = func(*args, **argv)
        with mutex:
            nonlocal n_thread
            n_thread.value +=1
            print(f"\r{n_thread.value} objects are processed...",end ='',flush = True)
        return result
    return wrapper

In [ ]:
with open('id_queries.pkl','rb') as f:
    id_queries = pickle.load(f)
with open('final_text.pkl','rb') as f:
    final_text = pickle.load(f)   

In [ ]:
id_urls = {}
with codecs.open('urls.numerate.txt') as f:
    for line in f:
        new_line = line[:-1].split('\t')
        id_urls[new_line[0]] = new_line[1]

In [ ]:
with open('url_docname.pkl','rb') as f:
    url_docname = pickle.load(f)
with open('docid_urls.pkl','rb') as f:
    id_urls = pickle.load(f)

In [ ]:
url_docname['kak-gta-san-andreas-sest-na-perednee-sidene.blogbuzz.pp.ru'] = 'content/content/20170702/doc.0087.dat'

In [ ]:
with open('id_docs.pkl','rb') as f:
    id_docs = pickle.load(f)

In [ ]:

@illustration
def w_1(ad):
        try:
            score_docs = defaultdict(lambda:defaultdict(dict))
            docs = id_docs[ad]
            query = id_queries[ad]
            words = query.split()
            l_r = len(words)
            dop_words = [' '.join([words[i],words[i+1]]) for i in range(len(words)-1)]
            words += dop_words
            phrase = {}
            for doc in docs:
                for word in query.split():
                    text = final_text[url_docname[id_urls[doc]][16:]]
                    n = len(re.findall(word,text))
                    d_l = len(text)
                    if ' ' in word:
                        w_s = word.split()
                        n += len(re.findall(' '.join(w_s[::-1]),text))*0.5
                        n += len(re.findall(w_s[0] + '\w+' +w_s[1],text))*0.5
                    score_docs[doc][word] = {'n':n,'dl':d_l}
                n = len(re.findall(query,text))
                phrase[doc] = {'n':n,'dl':d_l}
            cf = defaultdict(int)
            doc_s = defaultdict(float)
            for word in query.split():
                for doc in docs:
                    cf[word] += score_docs[doc][word]['n']
            for doc in docs:
                N = 0
                words_ps = 0
                for word in query.split():
                    if ' ' not in word:
                        if score_docs[doc][word]['n'] == 0:
                            N += 1
                        c = (-1.5)*cf[word]/score_docs[doc][word]['dl']
                        p = math.log(1 - math.exp(c) + 0.0001)
                        tf = score_docs[doc][word]['n']/score_docs[doc][word]['dl']
                        doc_s[doc] += p * tf/(tf + 1 + 1/350*score_docs[doc][word]['dl'])
                        words_ps += p
                    else:
                        w_s = word.split()
                        c_1 = (-1.5)*cf[w_s[0]]/score_docs[doc][w_s[0]]['dl']
                        p_1 = math.log(1 - math.exp(c_1))
                        c_2 = (-1.5)*cf[w_s[1]]/score_docs[doc][w_s[1]]['dl']
                        p_2 = math.log(1 - math.exp(c_2))
                        tf = score_docs[doc][word]['n']/score_docs[doc][word]['dl']
                        doc_s[doc] += 0.3*(p1 + p2) * tf/(tf + 1)
                        words_ps += p
                doc_s[doc] += 0.2*words_ps * 0.03**(l_r - N)
                tf = phrase[doc]['n']/phrase[doc]['dl']
                doc_s[doc] += 0.1*words_ps * tf/(tf + 1)
            res[ad] = doc_s
        except:
            mistakes.append(ad)
            print('!ERROR!',ad)

In [ ]:
def their_bm(ad):
    docs = id_docs[ad]
    corpus = []
    for doc in tqdm(docs):
        corpus.append(final_text[url_docname[id_urls[doc]][16:]].split())
    
    bm25 = BM25Okapi(corpus)   
    scoring = bm25.get_scores(id_queries[ad].split())
    return scoring

In [ ]:
res = {}
mistakes = []
with ThreadPool(10) as pool: 
    pool.map(w_1, range(1,400))

In [ ]:
with open('result_text_yandex.pkl','wb') as f:
    pickle.dump(res,f)

In [ ]:
for i in range(1,400):
    if i not in res.keys():
        print(i)

In [ ]:
# with open('sub.txt','w') as f:
#     f.write('QueryId,DocumentId\n')
#     for i in range(1,400):
#         for item in res[i]:
#             f.write('{},{}\n'.format(i,item[0]))